In [1]:
import signal
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings(action='ignore') #ignore all warnings include SSL warning for reqest.get
import time

In [2]:
#timeout
class WeirdLink(Exception):
    pass
def handler(signum, frame):
    print("timeOut!")
    raise WeirdLink
signal.signal(signal.SIGALRM, handler)

<Handlers.SIG_DFL: 0>

In [3]:
headers = {
    'Accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
}

In [4]:
def writeLink(LinksOut, text, linkUrl, url, errorFound):
    if not errorFound:
        LinksOut.write(url+'\n')
    LinksOut.write('\t'+text+'\n')
    LinksOut.write('\t'+linkUrl+'\n')
    LinksOut.write('\n')

In [5]:
#print error messages of every dead link under a given page
def printError(url,websiteName):
    deadLinksOut = open(websiteName+"DeadLinks.txt", "a")
    weirdLinksOut = open(websiteName+"WeirdLinks.txt", "a")
    errorFound = False
    try:
        req = requests.get(url, headers=headers, timeout = 30)
        req.raise_for_status()
    except Exception:
        return
    soup = BeautifulSoup(req.content, 'html.parser')
    content = soup.find_all("div", {"class": "region-content"})[0]
    for link in content.find_all('a'):
        linkUrl = link.get('href')
        text = link.get_text()
        if linkUrl == None:
            continue
        if "http" not in linkUrl:
            if "mailto:" or "tel:" in linkUrl:
                continue
            linkUrl = website + linkUrl
        try:
            signal.alarm(29)
            r = requests.get(linkUrl, headers=headers, verify=False, timeout = 30)
            signal.alarm(0)
        except WeirdLink:
            writeLink(deadLinksOut, text, linkUrl, url, errorFound)
            writeLink(weirdLinksOut, text, linkUrl, url, errorFound)
            errorFound = True
        except:
            writeLink(deadLinksOut, text, linkUrl, url, errorFound)
            errorFound = True
        try:
            r.raise_for_status()
        except Exception:
            writeLink(deadLinksOut, text, linkUrl, url, errorFound)
            errorFound = True
            # print('\t'+str(e))
    if errorFound:
       deadLinksOut.write('\n')
    deadLinksOut.close()
    weirdLinksOut.close()

In [6]:
website = "https://studentlife.umich.edu/parents"
req = requests.get(website+"/sitemap.xml", headers=headers, timeout = 30)
soup = BeautifulSoup(req.content, 'html.parser')
websiteName = website[website.find("//")+2:].replace("/","_")

In [7]:
urls = []
for link in soup.find_all('loc'):
    urls.append(link.string)

In [8]:
for i in tqdm_notebook(range(len(urls))):
    printError(urls[i],websiteName)

  0%|          | 0/428 [00:00<?, ?it/s]